In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, MaxPooling2D, BatchNormalization, Conv2D,Dropout,Flatten,BatchNormalization
from tensorflow.keras.preprocessing import  image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

In [3]:
import seaborn as sns
# import cv2
import matplotlib.image as png
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

In [4]:
data_directory = './PlantVillage'
# test_directory = '/content/drive/MyDrive/AI/Deep Learning/Datasets/Plant Datasets/Test/Test'
# valid_directory =  '/content/drive/MyDrive/AI/Deep Learning/Datasets/Plant Datasets/Validation/Validation'

In [5]:
train = image_dataset_from_directory(data_directory,batch_size=40,image_size=(256,256),shuffle=True)
# test =  image_dataset_from_directory(test_directory,batch_size=40,image_size=(256,256),shuffle=True)
# valid = image_dataset_from_directory(valid_directory,batch_size=40,image_size=(256,256),shuffle=True)

Found 20638 files belonging to 15 classes.


In [6]:
class_labels = train.class_names
class_labels

['Pepper__bell___Bacterial_spot',
 'Pepper__bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Tomato_Bacterial_spot',
 'Tomato_Early_blight',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite',
 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']

In [7]:
# for image_batch, image_label in train.take(1):
#   first_image = image_batch[0].numpy()
#   print()
#   total_pixel_value = tf.reduce_sum(first_image).numpy()
#   print("Total pixel value of the first image:", total_pixel_value)

In [8]:
# plt.figure(figsize=(20,20))
# for image_batch,image_label in train.take(1):
#   for i in range(20):
#     plt.subplot(5,4,i+1)
#     plt.imshow(image_batch[i].numpy().astype("uint8"))
#     plt.title(class_labels[image_label[i].numpy()])
#     plt.axis("off")


In [9]:
# data Augumentation and Normalization
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255,  # Normalize pixel values
    validation_split=0.6
)

val_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.12
)



In [10]:
train_generator = train_datagen.flow_from_directory(
    directory=data_directory,
    target_size=(256, 256),
    batch_size=100,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)


val_generator = val_datagen.flow_from_directory(
       directory=data_directory,
        batch_size=50,
        shuffle=False,
        class_mode='categorical',
        subset='validation'
  )



Found 8261 images belonging to 15 classes.
Found 2471 images belonging to 15 classes.


In [11]:
# def data_generator_worker(data_gen,subset,shuffle):
#         generator = data_gen.flow_from_directory(data_directory, target_size=(256, 256), batch_size=200, class_mode='categorical', subset=subset,shuffle=shuffle)
#         while True:        
#             yield next(generator)

In [12]:
# train_generator = data_generator_worker(train_datagen,'training',True)
# val_generator = data_generator_worker(val_datagen,'validation',False)

In [13]:
training_samples = 8261 
val_samples = 2471 
batch_size = 100
steps_per_epoch = training_samples // batch_size
validation_steps = val_samples // batch_size


In [14]:
base_model = VGG16(include_top=False,weights='imagenet',input_shape=(256,256,3))

In [15]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [16]:
base_model.trainable = False

In [17]:
# for layer in base_model.layers:
#    print(f'{layer.name} ===> {layer.trainable}')

In [18]:
model = Sequential()



model.add(base_model)
model.add(Conv2D(60,kernel_size=(3,3),strides=(2,2)))
model.add(MaxPooling2D((2,2),strides=(1,1)))
model.add(Conv2D(120,kernel_size=(3,3),strides=(2,2),padding='same'))
model.add(Conv2D(240,kernel_size=(3,3),strides=(2,2),padding='same'))
model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(236,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(15,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 conv2d (Conv2D)             (None, 3, 3, 60)          276540    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 60)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 1, 120)         64920     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 240)         259440    
                                                                 
 flatten (Flatten)           (None, 240)               0         
                                                        

In [19]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='loss',patience=5)

checkpoint = ModelCheckpoint(filepath='./Plant_Disease_Detection.h5',save_best_only=True,save_weights_only=False,monitor='loss')

In [ ]:
history = model.fit(train_generator,epochs=10,validation_data=val_generator,callbacks=[es,checkpoint,])

Epoch 1/10
83/83 [==============================] - 1310s 16s/step - loss: 2.1565 - accuracy: 0.2956 - val_loss: 1.3189 - val_accuracy: 0.5451
Epoch 2/10
83/83 [==============================] - 1410s 17s/step - loss: 1.4423 - accuracy: 0.5193 - val_loss: 0.9818 - val_accuracy: 0.6698
Epoch 3/10
83/83 [==============================] - 1424s 17s/step - loss: 1.1566 - accuracy: 0.6170 - val_loss: 0.7944 - val_accuracy: 0.7341
Epoch 4/10
83/83 [==============================] - 1432s 17s/step - loss: 1.0238 - accuracy: 0.6596 - val_loss: 0.6867 - val_accuracy: 0.7612
Epoch 5/10
12/83 [===>..........................] - ETA: 15:45 - loss: 0.9125 - accuracy: 0.7058

In [ ]:
# plt.plot(history.history['loss'], label='training loss')
# plt.plot(history.history['val_loss'], label='validation loss')
# plt.legend()
# plt.show()

In [ ]:
train_new_model()